# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Question 1

In [2]:
! pip install beautifulsoup4
! pip install lxml
! pip install html5lib
! pip install requests

import pandas as pd
import html
from bs4 import BeautifulSoup
import requests
import urllib.request


In [3]:
!wget -q -O 'Toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [115]:
with open('Toronto_data.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [116]:
#table = soup.find('table',class_='wikitable sortable')
table = soup.find('table')
#sum = table.td.text
#print(table)

In [117]:
column_names = ['Postcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [118]:
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [119]:
df=df[df.Borough != 'Not assigned']
#df=df[df['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [120]:
#df[df.Neighbourhood =='Not assigned']=df.Borough
df[df['Neighbourhood']=='Not assigned']=df['Borough']

In [122]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [129]:
temp_df=df.groupby('Postcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

In [131]:
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [133]:
df_merge = pd.merge(df, temp_df, on='Postcode')

In [134]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [135]:
df_merge.drop_duplicates(inplace=True)

In [136]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [137]:
df_merge.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,Queen's Park,Queen's Park,Queen's Park


In [139]:
df_merge.shape

(103, 3)